In [43]:
import numpy as np
import time

In [44]:
# -------------------- Rosenbrock Function --------------------
def rosenbrock(x):
    return np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

In [45]:
# -------------------- DE Parameters --------------------
D = 4                     # Dimension of solution vector
lower_bound = -5          # Rosenbrock lower bound
upper_bound = 5           # Rosenbrock upper bound

NP = 20                   # Population size
F = 0.8                   # Scaling factor
CR = 0.9                  # Crossover probability
max_generations = 1000    # Max generations
tol = 1e-6                # Tolerance for stopping


In [46]:
# -------------------- Initialization --------------------
np.random.seed(42)

pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([rosenbrock(ind) for ind in pop])

best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]

print(f"Generation 0: Best fitness = {best_fitness:.6e}, Best vector = {best_vector}")


Generation 0: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]


In [47]:
# -------------------- Start Timing --------------------
start_time = time.perf_counter()


In [48]:
# -------------------- DE Main Loop --------------------
generation = 0
fitness_history = [best_fitness]
while generation < max_generations and best_fitness > tol:
    generation += 1

    for i in range(NP):
        # Mutation
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, size=3, replace=False)

        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = np.clip(donor, lower_bound, upper_bound)

        # Crossover
        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            if np.random.rand() < CR or j == j_rand:
                trial[j] = donor[j]
            else:
                trial[j] = pop[i][j]

        # Selection
        f_trial = rosenbrock(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()

    fitness_history.append(best_fitness)

    print(f"Generation {generation:3d}: Best fitness = {best_fitness:.6e}, Best vector = {best_vector}")

    if best_fitness <= tol:
        print(f"\nTerminated at generation {generation} (fitness ≤ {tol}).")
        break


Generation   1: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   2: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   3: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   4: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   5: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   6: Best fitness = 2.002754e+03, Best vector = [-1.95757757  0.24756432 -0.68054981 -2.0877086 ]
Generation   7: Best fitness = 4.078390e+02, Best vector = [-1.62673855  0.89840294  1.65017866  2.23394896]
Generation   8: Best fitness = 4.078390e+02, Best vector = [-1.62673855  0.89840294  1.65017866  2.23394896]
Generation   9: Best fitness = 4.047601e+02, Best vector = [-0.35858485  0.48880911 -0.03217378 -1.9515565 ]
Generation  10: Bes

In [49]:
# -------------------- End Timing --------------------
end_time = time.perf_counter()
elapsed = end_time - start_time

In [50]:
# -------------------- Final Output --------------------
if best_fitness > tol:
    print(f"\nReached max generations ({max_generations}) without hitting fitness ≤ {tol}.")
print(f"\nFinal best fitness = {best_fitness:.6e}")
print(f"Final best vector = {best_vector}")
print(f"\n▶ Total elapsed time: {elapsed:.4f} seconds")



Final best fitness = 8.133746e-07
Final best vector = [0.99986504 0.99969016 0.99936864 0.99877335]

▶ Total elapsed time: 0.3604 seconds


In [51]:
# -------------------- Complexity Summary --------------------
print("\n--- Theoretical Complexity ---")
print("Time Complexity (per run):   O(NP × D × G_max)")
print("  • NP = population size")
print("  • D  = dimension (here = 4)")
print("  • G_max = number of generations until convergence")
print("Space Complexity:            O(NP × D)")
print("  • We store NP candidate vectors of dimension D each")


--- Theoretical Complexity ---
Time Complexity (per run):   O(NP × D × G_max)
  • NP = population size
  • D  = dimension (here = 4)
  • G_max = number of generations until convergence
Space Complexity:            O(NP × D)
  • We store NP candidate vectors of dimension D each


In [52]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=fitness_history,
    x=list(range(len(fitness_history))),
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue'),
    marker=dict(size=4)
))

fig.update_layout(
    title='Differential Evolution on Rosenbrock Function',
    xaxis_title='Generation',
    yaxis_title='Fitness Value',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
